In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt
%matplotlib inline
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import norm, sem
from scipy.interpolate import UnivariateSpline
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
from numpy.random import default_rng

import seaborn as sns
from pycebox.ice import ice, ice_plot
from matplotlib.cm import seismic

from sympy import *

In [2]:
x1,x2,x3,y = [],[],[],[]
N = 1000

for i in range(0,N):
    X1 = np.random.uniform(-1, 1, size = 1)
    X2 = np.random.uniform(-1, 1, size = 1)
    X3 = np.random.uniform(-1, 1, size = 1)
    Y = 0.2*X1 - 5*X2 + 10*X3

    x1.append(X1)
    x2.append(X2)
    x3.append(X3)    
    y.append(Y)

In [3]:
data = np.concatenate([x1,x2,x3,y], axis = 1)
df = pd.DataFrame(data, columns=['X1','X2','X3','Y'])

In [4]:
df

,X1,X2,X3,Y
0,0.971359,0.794225,0.082902,-2.947832
1,-0.846742,-0.227677,0.022969,1.198726
2,-0.010015,-0.560931,0.427951,7.082167
3,-0.529197,0.234439,0.152219,0.244158
4,0.404386,0.113278,-0.137100,-1.856514
...,...,...,...,...
995,-0.336843,-0.405127,0.761152,9.569787
996,0.633999,0.848730,-0.533719,-9.454042
997,0.424236,0.651025,0.733072,4.160438
998,-0.947632,0.373578,-0.591935,-7.976769


In [5]:
X = df[['X1','X2','X3']]
X1, X2, X3 = df.X1, df.X2, df.X3
y = df.Y

std = {}

In [6]:
minus_X, plus_X, minus_y, plus_y = train_test_split(X, y, test_size=.5, random_state=42)
minus_df = minus_X.join(minus_y)
plus_df = plus_X.join(plus_y)

In [7]:
def change_features(X_og,X,i,alpha,group):
    X_list = []
    y_list = []

    for j in range(len(X)):
        new_values = {}
        std[i] = np.std(X[i])
        og_value = X[i].iloc[j]
        if group == 'plus':
            new_value = og_value + alpha*std[i]
        if group == 'minus':
            new_value = og_value - alpha*std[i]
        new_values[i] = new_value
        X_list.append(new_values)
    X_new = pd.DataFrame(X_list)
    X_new = X_new.join(X.reset_index().drop(columns=[i,'index']))
    for x in range(len(X_new)):
        y_list.append(0.2*X_new.X1[x] - 5*X_new.X2[x] + 10*X_new.X3[x]) 
    y_new = pd.DataFrame(y_list, columns=['Y'])
    df_new = X_new.join(y_new).append(X_og).reset_index().drop(columns=['index'])
    return(df_new)

In [8]:
plus_df_X1 = change_features(minus_df, plus_X, 'X1', 0.05, 'plus')
minus_df_X1 = change_features(plus_df, minus_X, 'X1', 0.05, 'minus')
minus_df_X1

,X1,X2,X3,Y
0,0.966799,-0.613786,0.357413,6.836418
1,0.418007,-0.563745,-0.429185,-1.389522
2,-0.114514,0.350593,0.289005,1.114185
3,-0.364751,-0.414483,0.360251,5.601975
4,-0.920226,0.956150,0.596827,1.003472
...,...,...,...,...
995,0.829616,0.093702,0.353160,3.229009
996,0.449465,0.603772,-0.957461,-12.503575
997,0.779600,-0.272990,0.710087,8.621739
998,0.630706,0.340794,0.711977,5.541940


In [9]:
def cate(y, y_new):
    mean0 = np.mean(y)
    mean1 = np.mean(y_new)
    tao = mean1 - mean0
    return tao

In [10]:
minus_cate = {}
for x in minus_X:
    minus_df_new = change_features(minus_df, minus_X, x, 0.05, 'minus')
    c = cate(minus_y, minus_df_new.Y)
    minus_cate[x] = c
pd.DataFrame([minus_cate])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,X1,X2,X3
0,-0.002816,0.071174,-0.145716


In [11]:
plus_cate = {}
for x in plus_X:
    plus_df_new = change_features(minus_df, plus_X, x, 0.05, 'plus')
    c = cate(plus_y, plus_df_new.Y)
    plus_cate[x] = c
pd.DataFrame([plus_cate])

,X1,X2,X3
0,0.176425,0.103073,0.317594


### Cancer test data

In [90]:
X = pd.read_csv('cervical_cancer_risks/X.csv')
y = pd.read_csv('cervical_cancer_risks/y.csv')
df = X.join(y)

In [91]:
df

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Biopsy
0,18.0,4.0,15.0000,1.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,6.140845,5.816901,0.0,0.0,0.0,0.0,0.0
1,15.0,1.0,14.0000,1.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,6.140845,5.816901,0.0,0.0,0.0,0.0,0.0
2,34.0,1.0,16.9953,1.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,6.140845,5.816901,0.0,0.0,0.0,0.0,0.0
3,52.0,5.0,16.0000,4.0,1.0,37.0,37.0,1.0,3.00,0.0,...,0.0,0.0,0.0,6.140845,5.816901,1.0,0.0,1.0,0.0,0.0
4,46.0,3.0,21.0000,4.0,0.0,0.0,0.0,1.0,15.00,0.0,...,0.0,0.0,0.0,6.140845,5.816901,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,34.0,3.0,18.0000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,6.140845,5.816901,0.0,0.0,0.0,0.0,0.0
854,32.0,2.0,19.0000,1.0,0.0,0.0,0.0,1.0,8.00,0.0,...,0.0,0.0,0.0,6.140845,5.816901,0.0,0.0,0.0,0.0,0.0
855,25.0,2.0,17.0000,0.0,0.0,0.0,0.0,1.0,0.08,0.0,...,0.0,0.0,0.0,6.140845,5.816901,0.0,0.0,0.0,0.0,0.0
856,33.0,2.0,24.0000,2.0,0.0,0.0,0.0,1.0,0.08,0.0,...,0.0,0.0,0.0,6.140845,5.816901,0.0,0.0,0.0,0.0,0.0


In [80]:
bin_col = []
for i in X:
    if  len(X[i].unique())<4:
        bin_col.append(i)

In [81]:
bin_col

['Smokes',
 'Hormonal Contraceptives',
 'IUD',
 'STDs',
 'STDs:condylomatosis',
 'STDs:cervical condylomatosis',
 'STDs:vaginal condylomatosis',
 'STDs:vulvo-perineal condylomatosis',
 'STDs:syphilis',
 'STDs:pelvic inflammatory disease',
 'STDs:genital herpes',
 'STDs:molluscum contagiosum',
 'STDs:AIDS',
 'STDs:HIV',
 'STDs:Hepatitis B',
 'STDs:HPV',
 'Dx:Cancer',
 'Dx:CIN',
 'Dx:HPV',
 'Dx']

In [82]:
for i in range(len(bin_col)):
    print(X[bin_col[i]].value_counts())

0.000000    722
1.000000    123
0.145562     13
Name: Smokes, dtype: int64
1.000000    481
0.000000    269
0.641333    108
Name: Hormonal Contraceptives, dtype: int64
0.000000    658
0.112011    117
1.000000     83
Name: IUD, dtype: int64
0.000000    674
0.104914    105
1.000000     79
Name: STDs, dtype: int64
0.000000    709
0.058433    105
1.000000     44
Name: STDs:condylomatosis, dtype: int64
0.0    858
Name: STDs:cervical condylomatosis, dtype: int64
0.000000    749
0.005312    105
1.000000      4
Name: STDs:vaginal condylomatosis, dtype: int64
0.000000    710
0.057105    105
1.000000     43
Name: STDs:vulvo-perineal condylomatosis, dtype: int64
0.000000    735
0.023904    105
1.000000     18
Name: STDs:syphilis, dtype: int64
0.000000    752
0.001328    105
1.000000      1
Name: STDs:pelvic inflammatory disease, dtype: int64
0.000000    752
0.001328    105
1.000000      1
Name: STDs:genital herpes, dtype: int64
0.000000    752
0.001328    105
1.000000      1
Name: STDs:molluscum c

In [83]:
def change_binary(df, X_feat, alpha):
    # alpha: percentage of values to change
    # Only keep binary variables
    binary = [0,1]
    df = df[df[X_feat].isin(binary)].reset_index()
    
    df_X = df.iloc[:,:-1]
    # Choose random indices to change values of
    rand = []
    m = int(np.floor(len(df_X)*alpha))
    for i in range(m):
        rand.append(random.randint(0,max(df_X.index)))
    for d in rand:
        df_X[X_feat].loc[d] = np.abs(df_X[X_feat].loc[d] - 1)
    
    rf = RandomForestClassifier()
    rf.fit(df.iloc[:,:-1], df.Biopsy)
    y_new = pd.DataFrame(rf.predict(df_X), columns=['pred_y'])
    df_new = df_X.join(y_new).reset_index().drop(columns=['index'])
    return(df_new)

In [94]:
def change_continuous(df, X_feat, alpha, sig):
    # sig: value to change sigma by
    df_X = df.iloc[:,:-1]
    # Choose random indices to change values of
    std = np.std(df_X[X_feat])
    rand = []
    m = int(np.floor(len(df_X)*alpha))
    for i in range(m):
        rand.append(random.randint(0,max(df_X.index)))
    for d in rand:
        df_X[X_feat].loc[d] = df_X[X_feat].loc[d] + sig * std
    
    rf = RandomForestClassifier()
    rf.fit(df.iloc[:,:-1], df.Biopsy)
    y_new = pd.DataFrame(rf.predict(df_X), columns=['pred_y'])
    df_new = df_X.join(y_new).reset_index().drop(columns=['index'])
    return(df_new)

In [85]:
def cate(y, y_new):
    mean0 = np.mean(y)
    mean1 = np.mean(y_new)
    tao = mean1 - mean0
    return tao

In [95]:
X = df.iloc[:,:-1]
cate_values = {}
for X_feat in X:
    if X_feat in bin_col:
        df_new = change_binary(df, X_feat, alpha=0.5)
    else:
        df_new = change_continuous(df, X_feat, alpha=0.5, sig=1)
    c = cate(df.Biopsy, df_new.pred_y)
    cate_values[X_feat] = c
pd.DataFrame([cate_values])

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:HIV,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx
0,-0.008159,-0.011655,-0.01049,-0.01049,-0.004931,-0.001166,-0.001166,-0.000103,0.0,0.003374,...,0.006283,0.006283,0.006283,0.0,0.0,0.0,0.0,0.0,-0.001166,-0.006993
